In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam
from torch.nn import SmoothL1Loss
from tqdm.notebook import tqdm
import mlflow
import joblib
import matplotlib.pyplot as plt

In [2]:
mlflow.set_tracking_uri("file:///home/nick/Documents/code/fraud-detection-mlops/mlflow/mlruns")
experiment_name = "CreditCardFraudAutoencoder"
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///home/nick/Documents/code/fraud-detection-mlops/mlflow/mlruns/989291882614398324', creation_time=1738753630865, experiment_id='989291882614398324', last_update_time=1738753630865, lifecycle_stage='active', name='CreditCardFraudAutoencoder', tags={}>

In [3]:
processed_data_dir = '../data/processed'
engineered_data_dir = '../data/engineered'
model_dir = '../models'
plot_path = '../logs'

In [4]:
with mlflow.start_run():
    # Load the data
    X_train_scaled = pd.read_csv(os.path.join(processed_data_dir, 'X_train_scaled.csv'))
    X_val_scaled = pd.read_csv(os.path.join(processed_data_dir, 'X_val_scaled.csv'))
    X_test_scaled = pd.read_csv(os.path.join(processed_data_dir, 'X_test_scaled.csv'))
    y_train = pd.read_csv(os.path.join(processed_data_dir, 'y_train.csv'))
    y_val = pd.read_csv(os.path.join(processed_data_dir, 'y_val.csv'))

    # Log basic information about the dataset
    mlflow.log_param("num_rows_train", X_train_scaled.shape[0])
    mlflow.log_param("num_columns_train", X_train_scaled.shape[1])
    mlflow.log_param("num_rows_val", X_val_scaled.shape[0])
    mlflow.log_param("num_columns_val", X_val_scaled.shape[1])

    # Convert data to tensors
    train_tensor = torch.FloatTensor(X_train_scaled.values)
    val_tensor = torch.FloatTensor(X_val_scaled.values)
    test_tensor = torch.FloatTensor(X_test_scaled.values)

    # Filter non-fraud cases
    non_fraud_mask_train = (y_train == 0).values.flatten()
    non_fraud_mask_val = (y_val == 0).values.flatten()

    X_train_non_fraud = X_train_scaled[non_fraud_mask_train]
    X_val_non_fraud = X_val_scaled[non_fraud_mask_val]

    # Create datasets
    train_dataset = TensorDataset(torch.tensor(X_train_non_fraud.values, dtype=torch.float32))
    val_dataset = TensorDataset(torch.tensor(X_val_non_fraud.values, dtype=torch.float32))

    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    mlflow.log_param("device", str(device))

    # Define Autoencoder model
    class Autoencoder(nn.Module):
        def __init__(self, input_dim, encoding_dim):
            super().__init__()
            self.enc1 = nn.Linear(input_dim, 128)
            self.enc_act1 = nn.ReLU()
            self.enc2 = nn.Linear(128, 64)
            self.enc_bn = nn.BatchNorm1d(64)
            self.enc_act2 = nn.ReLU()
            self.enc3 = nn.Linear(64, encoding_dim)
            self.dropout = nn.Dropout(0.2)
            self.dec1 = nn.Linear(encoding_dim, 64)
            self.dec_act1 = nn.ReLU()
            self.dec2 = nn.Linear(64, 128)
            self.dec_act2 = nn.ReLU()
            self.dec3 = nn.Linear(128, input_dim)

        def forward(self, x):
            enc1_out = self.enc_act1(self.enc1(x))
            enc2_out = self.enc_act2(self.enc_bn(self.enc2(enc1_out)))
            encoding = self.enc3(enc2_out)
            drop = self.dropout(encoding)
            dec1_out = self.dec_act1(self.dec1(drop))
            dec2_in = dec1_out + enc2_out
            dec2_out = self.dec_act2(self.dec2(dec2_in))
            dec3_in = dec2_out + enc1_out
            output = self.dec3(dec3_in)
            return output

    # Initialize Autoencoder
    input_dim = X_train_scaled.shape[1]
    encoding_dim = 32
    autoencoder = Autoencoder(input_dim, encoding_dim).to(device)
    mlflow.log_param("input_dim", input_dim)
    mlflow.log_param("encoding_dim", encoding_dim)

    # Define Encoder
    class Encoder(nn.Module):
        def __init__(self, autoencoder):
            super(Encoder, self).__init__()
            self.enc1 = autoencoder.enc1
            self.enc_act1 = autoencoder.enc_act1
            self.enc2 = autoencoder.enc2
            self.enc_bn = autoencoder.enc_bn
            self.enc_act2 = autoencoder.enc_act2
            self.enc3 = autoencoder.enc3

        def forward(self, x):
            x = self.enc_act1(self.enc1(x))
            x = self.enc_act2(self.enc_bn(self.enc2(x)))
            x = self.enc3(x)
            return x

    # Training setup
    lr = 1e-4
    criterion = SmoothL1Loss()
    optimizer = Adam(autoencoder.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)
    epochs = 100
    batch_size = 256
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count())
    val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=os.cpu_count())

    mlflow.log_param("learning_rate", lr)
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("batch_size", batch_size)

    # Training loop
    train_losses = []
    val_losses = []
    best_loss = float('inf')
    save_path = os.path.join(model_dir, 'best_autoencoder.pth')

    epoch_bar = tqdm(range(epochs), desc='Training Autoencoder', unit='epoch')
    for epoch in epoch_bar:
        # Training phase
        autoencoder.train()
        epoch_train_loss = 0
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}', leave=False, unit='batch'):
            batch = batch[0].to(device)
            optimizer.zero_grad()
            outputs = autoencoder(batch)
            loss = criterion(outputs, batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(autoencoder.parameters(), 1.0)
            optimizer.step()
            epoch_train_loss += loss.item() * batch.size(0)

        # Validation phase
        autoencoder.eval()
        epoch_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch[0].to(device)
                outputs = autoencoder(batch)
                loss = criterion(outputs, batch)
                epoch_val_loss += loss.item() * batch.size(0)

        # Calculate metrics
        train_loss = epoch_train_loss / len(train_loader.dataset)
        val_loss = epoch_val_loss / len(val_loader.dataset)
        train_losses.append(train_loss)
        val_losses.append(val_loss)

        # Update progress bar
        epoch_bar.set_postfix({
            'Train Loss': f'{train_loss:.4f}',
            'Val Loss': f'{val_loss:.4f}',
            'LR': f'{optimizer.param_groups[0]["lr"]:.2e}'
        })

        # Log metrics
        mlflow.log_metric("train_loss", train_loss, step=epoch)
        mlflow.log_metric("val_loss", val_loss, step=epoch)

        # Save best model
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(autoencoder.state_dict(), save_path)
            mlflow.log_artifact(save_path)

        # Learning rate scheduling
        scheduler.step(val_loss)

    # Plot training history
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Autoencoder Training Progress')
    plt.xlabel('Epochs')
    plt.ylabel('MSE Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig(plot_path)
    mlflow.log_artifact(plot_path)
    plt.close()

    # Load best model
    autoencoder.load_state_dict(torch.load(save_path))
    autoencoder.eval()
    encoder = Encoder(autoencoder).eval()

    # Generate encoded features
    with torch.no_grad():
        encoded_train = encoder(train_tensor.to(device)).cpu().numpy()
        encoded_val = encoder(val_tensor.to(device)).cpu().numpy()
        encoded_test = encoder(test_tensor.to(device)).cpu().numpy()

    # Save encoded features
    pd.DataFrame(encoded_train).to_csv(os.path.join(engineered_data_dir, 'encoded_features_train.csv'), index=False)
    pd.DataFrame(encoded_val).to_csv(os.path.join(engineered_data_dir, 'encoded_features_val.csv'), index=False)
    pd.DataFrame(encoded_test).to_csv(os.path.join(engineered_data_dir, 'encoded_features_test.csv'), index=False)

    # Enrich original features with encoded features
    original_features_train = pd.read_csv(os.path.join(processed_data_dir, 'X_train_scaled.csv'))
    encoded_features_train = pd.read_csv(os.path.join(engineered_data_dir, 'encoded_features_train.csv'))
    enriched_data_train = pd.concat([original_features_train, encoded_features_train], axis=1)
    original_features_val = pd.read_csv(os.path.join(processed_data_dir, 'X_val_scaled.csv'))
    encoded_features_val = pd.read_csv(os.path.join(engineered_data_dir, 'encoded_features_val.csv'))
    enriched_data_val = pd.concat([original_features_val, encoded_features_val], axis=1)
    original_features_test = pd.read_csv(os.path.join(processed_data_dir, 'X_test_scaled.csv'))
    encoded_features_test = pd.read_csv(os.path.join(engineered_data_dir, 'encoded_features_test.csv'))
    enriched_data_test = pd.concat([original_features_test, encoded_features_test], axis=1)

    # Save enriched data
    enriched_data_train.to_csv(os.path.join(engineered_data_dir, 'X_train_enriched.csv'), index=False)
    enriched_data_val.to_csv(os.path.join(engineered_data_dir, 'X_val_enriched.csv'), index=False)
    enriched_data_test.to_csv(os.path.join(engineered_data_dir, 'X_test_enriched.csv'), index=False)

    # Log final dataset shapes
    mlflow.log_metric("enriched_train_samples", enriched_data_train.shape[0])
    mlflow.log_metric("enriched_val_samples", enriched_data_val.shape[0])
    mlflow.log_metric("enriched_test_samples", enriched_data_test.shape[0])

print("MLflow run completed.")

Training Autoencoder:   0%|          | 0/100 [00:00<?, ?epoch/s]

Epoch 1:   0%|          | 0/756 [00:00<?, ?batch/s]

Epoch 2:   0%|          | 0/756 [00:00<?, ?batch/s]

Epoch 3:   0%|          | 0/756 [00:00<?, ?batch/s]

Epoch 4:   0%|          | 0/756 [00:00<?, ?batch/s]

...

Epoch 97:   0%|          | 0/756 [00:00<?, ?batch/s]

Epoch 98:   0%|          | 0/756 [00:00<?, ?batch/s]

Epoch 99:   0%|          | 0/756 [00:00<?, ?batch/s]

Epoch 100:   0%|          | 0/756 [00:00<?, ?batch/s]

MLflow run completed.
